In [24]:
import pandas as pd

In [25]:
#Reading in a short story as text sample into Python
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
          raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20398
﻿I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no


### 1. Tokenizing Text

In [27]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [28]:
#separting punction and spaces from the text
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [29]:
#remvoing white spaces
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [30]:
#extending the tokenizer to handle wider range of punctuations
text = "Hello, world. Is this-- a test."
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print("Tokenized text: ",result)

Tokenized text:  ['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '.']


In [31]:
#applying this tokenizer to the full Edith Warton's story
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f"length of the complete tokenized text from Edith Wharton story: {len(preprocessed)}")

length of the complete tokenized text from Edith Wharton story: 4690


In [32]:
#printing first 30 tokens
print(f"First 30 tokens: {preprocessed[:30]}")

First 30 tokens: ['\ufeffI', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2. Converting token into token IDs

#### create a list of all unique tokens and sort them alphabetically to determine the vocabulary

In [35]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(f"Vocabulary Size: {vocab_size}")

Vocabulary Size: 1131


In [36]:
#creating a vocabulary
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [37]:
#implementing a tokenizer class to tokenize text into tokens, encode tokens to integers and decode integers to tokens

class SimpleTokenzierV1:
    def __init__(self, vocab):
        self.str_to_int = vocab #A
        self.int_to_str = {i:s for s,i in vocab.items()} #B

    def encode(self, text): #C
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): #D
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #E
        return text

In [38]:
#instantiating tokenizer object to test the SimpleTokenzierV1 class
tokenizer = SimpleTokenzierV1(vocab)

In [39]:
text = raw_text[:99]
ids = tokenizer.encode(text)
print(f"Token ids for sample text from Edith Warton's story: {ids}")

Token ids for sample text from Edith Warton's story: [1130, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6, 1002, 115, 500, 435, 392, 6, 908, 585, 1077, 709]


In [40]:
#testing decoder on the token ids above
print(f"text from decoding token ids using SimpleTokenzierV1 class: \n{tokenizer.decode(ids)}")

text from decoding token ids using SimpleTokenzierV1 class: 
﻿I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was no


In [41]:
#running the tokenizer on a sample text which is not a part of the text used to create the vocab
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

here, the key error suggests that Hello is not a part of the vocabulary and hence we need to use large and diverse text in order to extend the vocabulary when creating a LARGE language models

#### 3. Adding special context tokens

modifying SimpleTokenzierV1 to support new tokens for unknown words and document boundaries

In [42]:
'''adding tokens:
1. <unk> - to represent unknown or new words that are not part of the vocabulary
2. <|endoftext|> -  marker to separate two different text source from each other
'''
all_tokens =  sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<unk>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

print(f"length of the vocabulary after extending the vocab with unknown words and end of text markers: {len(vocab.items())}")

length of the vocabulary after extending the vocab with unknown words and end of text markers: 1133


In [44]:
#printing a sample from the end of the extended vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('your', 1128)
('yourself', 1129)
('\ufeffI', 1130)
('<|endoftext|>', 1131)
('<unk>', 1132)


In [46]:
#SimpleTokenzierV2 replaces unknown words with the token "<unk>"
class SimpleTokenzierV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                       else "<unk>" for item in preprocessed] #A
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #B
        return text

In [48]:
#concatenating two unrelated texts
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))
print(f"concatenated text with endoftext marker: \n{text}")

concatenated text with endoftext marker: 
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [50]:
#testing the SimpleTokenzierV2 on a text that is concatenation of two unrelated texts
tokenizer = SimpleTokenzierV2(vocab)
print(tokenizer.encode(text))

[1132, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1132, 7]


we can see that the list of token IDs contains 1131 for the <|endoftext|> separator token as well as two 1132 tokens, which are used for unknown words.